In [26]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os
API_KEY = "" # Add your API key here

# Base URL for Polygon Forex API
BASE_URL = "https://api.polygon.io"

def fetch_paginated_forex_data(pair, multiplier, timespan, start_date, end_date):
    """
    Fetch all forex candlestick data from Polygon API, handling pagination.

    :param pair: Forex pair, e.g., 'C:EURUSD', 'C:EURGBP'
    :param multiplier: Time multiplier, e.g., 1 for 1-minute
    :param timespan: Timespan, e.g., 'minute', 'hour'
    :param start_date: Start date in 'YYYY-MM-DD' format
    :param end_date: End date in 'YYYY-MM-DD' format
    :return: DataFrame with all paginated candlestick data
    """
    final_df = pd.DataFrame()
    current_date = start_date

    while current_date < end_date:
        url = f"{BASE_URL}/v2/aggs/ticker/{pair}/range/{multiplier}/{timespan}/{current_date}/{end_date}"
        params = {
            "adjusted": "true",
            "sort": "asc",
            "limit": 50000,
            "apiKey": API_KEY
        }

        print(f"Fetching data from {current_date} to {end_date} for {pair}...")
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json().get("results", [])
            if not data:
                print("No more data available.")
                break

            df = pd.DataFrame(data)
            df.rename(columns={"t": "from", "o": "open", "h": "max", "l": "min", "c": "close", "v": "volume"}, inplace=True)
            df = df[["from", "open", "max", "min", "close", "volume"]]
            df['at'] = df['from']
            df['to'] = df['from']
            df['from'] = df['from'] // 1000
            df['to'] = df['to'] // 1000

            final_df = pd.concat([final_df, df], ignore_index=True)

            last_timestamp = df["to"].iloc[-1]
            last_current = current_date
            current_date = (datetime.fromtimestamp(last_timestamp) + timedelta(minutes=1)).strftime("%Y-%m-%d")
            if current_date == last_current:
                print("No more data available.")
                break
        elif response.status_code == 429:
            print("Rate limit exceeded. Waiting for 60 seconds...")
            time.sleep(60)
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break

    return final_df

def fetch_data_for_assets(assets, multiplier=1, timespan="minute", start_date="2020-01-01", end_date=None):
    """
    Fetches forex data for a list of assets and saves each dataset immediately.

    :param assets: List of asset symbols in Polygon format, e.g., ['C:EURUSD', 'C:EURGBP']
    :param multiplier: Time multiplier, e.g., 1 for 1-minute
    :param timespan: Timespan, e.g., 'minute', 'hour'
    :param start_date: Start date in 'YYYY-MM-DD' format
    :param end_date: End date in 'YYYY-MM-DD' format (defaults to today)
    """
    if end_date is None:
        end_date = datetime.now().strftime("%Y-%m-%d")
    
    for asset in assets:
        df = fetch_paginated_forex_data(asset, multiplier, timespan, start_date, end_date)
        if len(df) > 0:
            save_data_to_csv(asset, df)
            print(f"Fetched and saved {len(df)} rows for {asset}.")

def save_data_to_csv(asset, df, timespan="M1"):
    """
    Saves forex data to a CSV file.

    :param asset: Asset symbol
    :param df: DataFrame containing the asset's data
    :param timespan: Timeframe label for filename
    """
    if len(df) != 0:
        asset_name = asset.replace("C:", "").replace(":", "")
        df.to_csv(f"{asset_name}_{timespan}.csv", index=False)
        print(f"Saved {asset_name}_{timespan}.csv")

# Example usage
assets = [
    # # Forex Currency Pairs
    "C:EURUSD", "C:USDJPY", "C:GBPUSD", "C:AUDUSD", "C:USDCHF",
    "C:USDCAD", "C:NZDUSD", "C:EURGBP", "C:EURJPY", "C:GBPJPY",
    "C:AUDJPY", "C:EURAUD", "C:CHFJPY", "C:EURCAD", "C:GBPCAD",
    "C:AUDCAD", "C:NZDJPY", "C:GBPCHF", "C:EURCHF", "C:USDSGD",

    # # Commodities
    "C:XAUUSD", "C:XAGUSD",

    # # Cryptocurrencies
    "X:BTCUSD", "X:ETHUSD", "X:LTCUSD", "X:XRPUSD",

    # # Indices
    "AAPL", "MSFT", "AMZN", "GOOGL"
]

fetch_data_for_assets(assets)

Fetching data from 2020-01-01 to 2025-03-26 for AAPL...
Fetching data from 2023-07-03 to 2025-03-26 for AAPL...
Fetching data from 2023-10-04 to 2025-03-26 for AAPL...
Fetching data from 2024-01-12 to 2025-03-26 for AAPL...
Fetching data from 2024-04-17 to 2025-03-26 for AAPL...
Fetching data from 2024-07-22 to 2025-03-26 for AAPL...
Rate limit exceeded. Waiting for 60 seconds...
Fetching data from 2024-07-22 to 2025-03-26 for AAPL...
Fetching data from 2024-10-24 to 2025-03-26 for AAPL...
Fetching data from 2025-02-04 to 2025-03-26 for AAPL...
Fetching data from 2025-03-25 to 2025-03-26 for AAPL...
No more data available.
Saved AAPL_M1.csv
Fetched and saved 376540 rows for AAPL.
Fetching data from 2020-01-01 to 2025-03-26 for MSFT...
Fetching data from 2023-07-19 to 2025-03-26 for MSFT...
Rate limit exceeded. Waiting for 60 seconds...
Fetching data from 2023-07-19 to 2025-03-26 for MSFT...
Fetching data from 2023-11-08 to 2025-03-26 for MSFT...
Fetching data from 2024-03-01 to 2025-03